In [1]:
%pip install requests aria2 netCDF4 numpy xarray scikit-learn tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 18.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 26.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 35.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 30.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 47.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

# Constants
DOWNLOAD_DATA = True
DATA_DIR = './data'  # Directory containing .tar.gz files
EXTRACT_DIR = os.path.join(DATA_DIR, 'extracted')

# Temporary file for download links
TMP_FILE = os.path.join(DATA_DIR, 'tmp.txt')
EXTRACT_DIR = os.path.join(DATA_DIR, 'extracted')

# Bucket and endpoint configuration
CUSTOM_ENDPOINT = "bbproxy.meyerstk.com/file"
APP = "TorNetBecauseZenodoSlow"  # Bucket name

In [3]:
import logging
import subprocess
import tarfile

# Setup logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")

# Ensure directories exist
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(EXTRACT_DIR, exist_ok=True)


def download_links(links):
    """
    Download files from the provided links using aria2c.
    Uses a file named tmp.txt in DATA_DIR for links.
    """
    try:
        # Write links to tmp.txt
        with open(TMP_FILE, 'w') as file:
            file.writelines(link + '\n' for link in links)
        logging.info(f"Temporary file created: {TMP_FILE}")

        # Run aria2c to download files
        logging.info(f"Starting downloads for links: {', '.join(links)}")
        command = [
            "aria2c",
            "-j", "5",                # Download up to 3 files concurrently
            "-x", "16",               # Use up to 16 connections per file
            # "--console-log-level=info",
            "-s", "16",               # Split each file into 16 segments
            "--dir", DATA_DIR,        # Specify the download directory
            "-i", TMP_FILE            # Input file with download links
        ]
        subprocess.run(command, check=True)
        logging.info("Downloads completed successfully.")
    except Exception as e:
        logging.error(f"Error during download: {e}")
        exit(1)
    finally:
        if os.path.exists(TMP_FILE):
            os.remove(TMP_FILE)
            logging.info(f"Temporary file deleted: {TMP_FILE}")


def download_files_with_aria():
    """
    Download files from a public Backblaze B2 bucket served via a custom endpoint using aria2c.
    """
    logging.info("Starting download process with aria2c...")

    # # List of files to download
    file_list = [
        "tornet_2013.tar.gz",
        # "tornet_2014.tar.gz",
        # "tornet_2015.tar.gz",
        # "tornet_2016.tar.gz",
        # "tornet_2017.tar.gz",
        # "tornet_2018.tar.gz",
        # "tornet_2019.tar.gz",
        # "tornet_2020.tar.gz",
        # "tornet_2021.tar.gz",
        # "tornet_2022.tar.gz",
        "catalog.csv"
    ]

    # Construct the public URLs
    links = [f"https://{CUSTOM_ENDPOINT}/{APP}/{file_name}" for file_name in file_list]

    links = [
        "https://zenodo.org/records/12636522/files/catalog.csv",
        "https://zenodo.org/records/12636522/files/tornet_2013.tar.gz",
    ]
    
    # Filter out already downloaded files
    links_to_download = [
        link for link in links
        if not os.path.exists(os.path.join(DATA_DIR, os.path.basename(link)))
    ]

    if links_to_download:
        download_links(links_to_download)
    else:
        logging.info("All files already downloaded.")


def extract_local_tar_files():
    """
    Extract all .tar.gz files from the local DATA_DIR to EXTRACT_DIR.
    """
    logging.info("Starting extraction process...")
    for file_name in os.listdir(DATA_DIR):
        if file_name.endswith('.tar.gz'):
            file_path = os.path.join(DATA_DIR, file_name)
            logging.info(f'Extracting {file_path}...')
            with tarfile.open(file_path, 'r:gz') as tar:
                tar.extractall(path=EXTRACT_DIR)
            logging.info(f'Extracted {file_path} to {EXTRACT_DIR}')

# if DOWNLOAD_DATA:
#     download_files_with_aria()

# Call the function to process the local .tar.gz files
extract_local_tar_files()

2024-12-05 00:22:59,454 - INFO - Starting download process with aria2c...
2024-12-05 00:22:59,456 - INFO - Temporary file created: ./data/tmp.txt
2024-12-05 00:22:59,457 - INFO - Starting downloads for links: https://bbproxy.meyerstk.com/file/TorNetBecauseZenodoSlow/tornet_2013.tar.gz, https://bbproxy.meyerstk.com/file/TorNetBecauseZenodoSlow/catalog.csv



12/05 00:22:59 [NOTICE] Downloading 2 item(s)
[DL:0B][#70f286 0B/0B][#ad92d6 0B/0B]
[DL:0B][#70f286 0B/0B][#ad92d6 0B/0B]
[DL:0B][#70f286 0B/0B][#ad92d6 0B/0B]
[DL:0B][#70f286 0B/0B][#ad92d6 0B/0B]
[DL:0B][#70f286 0B/0B][#ad92d6 0B/0B]
[DL:1.0MiB][#70f286 496KiB/2.9GiB(0%)][#ad92d6 112KiB/36MiB(0%)]
[DL:11MiB][#70f286 12MiB/2.9GiB(0%)][#ad92d6 6.5MiB/36MiB(17%)]
[DL:50MiB][#70f286 110MiB/2.9GiB(3%)][#ad92d6 21MiB/36MiB(58%)]
[DL:103MiB][#70f286 336MiB/2.9GiB(11%)][#ad92d6 35MiB/36MiB(97%)]

12/05 00:23:10 [NOTICE] Download complete: ./data/catalog.csv
[#70f286 469MiB/2.9GiB(15%) CN:16 DL:102MiB ETA:24s]
[#70f286 825MiB/2.9GiB(27%) CN:16 DL:148MiB ETA:14s]
[#70f286 1.0GiB/2.9GiB(36%) CN:16 DL:165MiB ETA:11s]
[#70f286 1.3GiB/2.9GiB(44%) CN:16 DL:178MiB ETA:9s]
[#70f286 1.5GiB/2.9GiB(53%) CN:16 DL:187MiB ETA:7s]
[#70f286 1.7GiB/2.9GiB(60%) CN:16 DL:192MiB ETA:6s]
[#70f286 2.0GiB/2.9GiB(68%) CN:16 DL:218MiB ETA:4s]
[#70f286 2.2GiB/2.9GiB(77%) CN:16 DL:242MiB ETA:2s]
[#70f286 2.4GiB/2.9GiB

2024-12-05 00:23:59,931 - INFO - Downloads completed successfully.
2024-12-05 00:23:59,933 - INFO - Temporary file deleted: ./data/tmp.txt
2024-12-05 00:23:59,935 - INFO - Starting extraction process...
2024-12-05 00:23:59,937 - INFO - Extracting ./data/tornet_2013.tar.gz...



12/05 00:23:59 [NOTICE] Download complete: ./data/tornet_2013.tar.gz

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
ad92d6|OK  |    10MiB/s|./data/catalog.csv
70f286|OK  |   174MiB/s|./data/tornet_2013.tar.gz

Status Legend:
(OK):download completed.


2024-12-05 00:24:12,499 - INFO - Extracted ./data/tornet_2013.tar.gz to ./data/extracted


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from pathlib import Path
import xarray as xr

# Constants
DATA_DIR = './data'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')
TFRECORD_DIR = os.path.join(DATA_DIR, 'tfrecords')
BATCH_SIZE = 32
BUFFER_SIZE = 10000
CHANNEL_MIN_MAX = {
    'DBZ': [-20., 60.],
    'VEL': [-60., 60.],
    'KDP': [-2., 5.],
    'RHOHV': [0.2, 1.04],
    'ZDR': [-1., 8.],
    'WIDTH': [0., 9.]
}

# Ensure TFRecord directory exists
os.makedirs(TFRECORD_DIR, exist_ok=True)

# Parsing and Preprocessing Functions
def parse_nc_file(file_path):
    """
    Parse and preprocess a single .nc file into normalized data and labels.
    """
    if not isinstance(file_path, str):
        file_path = file_path.numpy().decode("utf-8")  # Convert TensorFlow tensor to string

    try:
        with xr.open_dataset(file_path, engine="netcdf4") as ds:
            variables = ['DBZ', 'VEL', 'KDP', 'RHOHV', 'ZDR', 'WIDTH']
            data_list = []
            for var in variables:
                var_data = ds[var].values  # Shape: [time, azimuth, range, sweep]
                var_min, var_max = CHANNEL_MIN_MAX[var]
                var_data = np.nan_to_num(var_data, nan=0, posinf=0, neginf=0)  # Replace NaNs
                var_data = np.clip(var_data, var_min, var_max)  # Clip to range
                var_data = (var_data - var_min) / (var_max - var_min)  # Normalize to [0, 1]
                data_list.append(var_data)

            # Concatenate variables along the channel dimension
            data = np.stack(data_list, axis=-1)  # Shape: [time, azimuth, range, sweep, num_variables]

            # Reshape to combine sweep and variable dimensions into one channel dimension
            time, azimuth, range_, sweep, num_variables = data.shape
            num_channels = num_variables * sweep
            data = data.reshape(time, azimuth, range_, num_channels)

            # Extract the label
            label = ds.attrs.get("category", "NUL")
            label = 1 if label == "TOR" else 0

            return data, label

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None, None

def serialize_example(data, label):
    """
    Serialize data and label into TFRecord format.
    """
    feature = {
        "feature": tf.train.Feature(bytes_list=tf.train.BytesList(value=[data.tobytes()])),
        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
    }
    return tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()

def preprocess_to_tfrecord(files, output_path):
    """
    Preprocess a list of .nc files and save them as a single TFRecord file.
    """
    with tf.io.TFRecordWriter(output_path) as writer:
        for file in tqdm(files, desc=f"Processing {output_path}"):
            try:
                file_path = str(file)  # Convert PosixPath to string
                data, label = parse_nc_file(file_path)
                if data is not None:
                    serialized_example = serialize_example(data, label)
                    writer.write(serialized_example)
            except Exception as e:
                print(f"Failed to process {file}: {e}")

def create_tfrecords(input_dir, output_dir):
    """
    Create TFRecord files from .nc files organized by year.
    """
    os.makedirs(output_dir, exist_ok=True)
    years = [p for p in Path(input_dir).iterdir() if p.is_dir()]

    for year_dir in years:
        year = year_dir.name
        files = list(year_dir.rglob("*.nc"))
        files = [str(f) for f in files]  # Convert PosixPath to string
        output_path = os.path.join(output_dir, f"{year}.tfrecord")

        print(f"Processing year {year} with {len(files)} files...")
        preprocess_to_tfrecord(files, output_path)

# Dataset Loading
def parse_tfrecord(serialized_example):
    """
    Parse a serialized TFRecord example.
    """
    feature_description = {
        "feature": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(serialized_example, feature_description)
    data = tf.io.decode_raw(example["feature"], tf.float32)
    data = tf.reshape(data, [4, 120, 240, 12])  # Shape derived from input dimensions
    label = example["label"]
    return data, label

def permute_axes(features, label):
    """
    Rearrange axes of the features to match model input shape.
    Input: (time, azimuth, range, channels)
    Output: (azimuth, range, channels, time)
    """
    features = tf.transpose(features, perm=[1, 2, 3, 0])  # Move 'time' to the last axis
    return features, label

def create_tf_dataset_from_tfrecord(tfrecord_dir, batch_size=32, buffer_size=10000):
    """
    Create a tf.data.Dataset from TFRecord files with enhanced parallelism.
    """
    tfrecord_files = tf.data.Dataset.list_files(f"{tfrecord_dir}/*.tfrecord", shuffle=True)
    dataset = tfrecord_files.interleave(
        lambda x: tf.data.TFRecordDataset(x),
        cycle_length=16,  # Number of parallel files to read
        num_parallel_calls=tf.data.AUTOTUNE
    )
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(permute_axes, num_parallel_calls=tf.data.AUTOTUNE)  # Adjust axis order
    dataset = dataset.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset


print("Processing training data...")
create_tfrecords(TRAIN_DIR, train_tfrecord_dir)

print("Processing testing data...")
create_tfrecords(TEST_DIR, test_tfrecord_dir)

# Step 2: Load datasets from TFRecords
print("Loading training dataset...")
train_dataset = create_tf_dataset_from_tfrecord(train_tfrecord_dir, batch_size=BATCH_SIZE)

print("Loading testing dataset...")
test_dataset = create_tf_dataset_from_tfrecord(test_tfrecord_dir, batch_size=BATCH_SIZE)

# Step 3: Validate the pipeline
print("Validating dataset...")
for data, label in test_dataset.take(1):
    print(f"Data shape: {data.shape}")
    print(f"Label: {label}")

Processing training data...
Processing year 2013 with 3498 files...


Processing ./data/tfrecords/train/2013.tfrecord:  86%|████████▌ | 2993/3498 [02:10<00:21, 23.06it/s]

In [5]:
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.regularizers import l2


def create_3d_torcnn(input_shape=(120, 240, 36, 3), dropout_rate=0.3):
    """
    Define a 3D CNN model for tornado detection.
    """
    model = models.Sequential(
        [
            # Block 1
            layers.Input(shape=input_shape),
            layers.Conv3D(32, (3, 3, 3), activation="relu", padding="same"),
            layers.Conv3D(32, (3, 3, 3), activation="relu", padding="same"),
            layers.BatchNormalization(),
            layers.MaxPooling3D((2, 2, 1)),  # Pool across spatial dimensions only
            layers.Dropout(dropout_rate),
            # Block 2
            layers.Conv3D(64, (3, 3, 3), activation="relu", padding="same"),
            layers.Conv3D(64, (3, 3, 3), activation="relu", padding="same"),
            layers.BatchNormalization(),
            layers.MaxPooling3D(
                (2, 2, 2)
            ),  # Pool across spatial and temporal dimensions
            layers.Dropout(dropout_rate),
            # Block 3
            layers.Conv3D(128, (3, 3, 3), activation="relu", padding="same"),
            layers.Conv3D(128, (3, 3, 3), activation="relu", padding="same"),
            layers.BatchNormalization(),
            layers.MaxPooling3D((2, 2, 2)),
            layers.Dropout(dropout_rate),
            # Block 4
            layers.Conv3D(256, (3, 3, 3), activation="relu", padding="same"),
            layers.BatchNormalization(),
            layers.MaxPooling3D((2, 2, 2)),
            layers.Dropout(dropout_rate),
            # Fully Connected Layers
            layers.Flatten(),
            layers.Dense(128, activation="relu", kernel_regularizer=l2(0.01)),
            layers.BatchNormalization(),
            layers.Dropout(0.4),
            layers.Dense(64, activation="relu", kernel_regularizer=l2(0.01)),
            layers.BatchNormalization(),
            layers.Dropout(0.4),
            # Output Layer
            layers.Dense(1, activation="sigmoid"),
        ]
    )

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss=BinaryCrossentropy(),
        metrics=["accuracy", Precision(), Recall(), AUC()],
    )
    return model


# Create Model
# Note: Add the temporal dimension to the input shape (TIME_STEPS = 3).
input_shape = (120, 240, len(VARIABLES) * SWEEPS, TIME_STEPS)
model = create_3d_torcnn(input_shape=input_shape)

In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
# import matplotlib.pyplot as plt

# Hyperparameters
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.0005
DROPOUT_RATE = 0.3

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,  # Reduce learning rate by a factor of 0.5
    patience=3,  # Wait 3 epochs of no improvement before reducing
    min_lr=1e-6,  # Lower bound for the learning rate
    verbose=1  # Print updates when learning rate is reduced
)

# Start Training
print("Starting model training...")
history = model.fit(
    train_dataset,  # Training dataset with features and labels
    epochs=EPOCHS,
    validation_data=test_dataset,  # Validation dataset with features and labels
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the Model
print("Evaluating the model...")
results = model.evaluate(X_test)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")

# Extract Features and Labels for Detailed Metrics
X_test_features = []
y_test_labels = []

for features, labels in X_test:
    X_test_features.append(features.numpy())
    y_test_labels.append(labels.numpy())

X_test_features = np.concatenate(X_test_features, axis=0)
y_test_labels = np.concatenate(y_test_labels, axis=0)

# Predictions
y_pred = (model.predict(X_test_features) > 0.5).astype(int)

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test_labels, y_pred))

# Classification Report
print("Classification Report:")
print(classification_report(y_test_labels, y_pred))

# ROC Curve
fpr, tpr, _ = roc_curve(y_test_labels, model.predict(X_test_features))
# plt.figure()
# plt.plot(fpr, tpr, label='ROC Curve')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve')
# plt.legend(loc='lower right')
# plt.show()


Starting model training...
Epoch 1/50


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "conv3d" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (None, 4, 120, 240, 12)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 4, 120, 240, 12), dtype=float32)
  • training=True
  • mask=None